# Uniswap V3 price analysis

In this notebook we will show how to download price events from Uniswap V3 to your computer as CSV files and use them to analyse price in each pool.

You will need to havev Ethereum JSON-RPC API node URL in order to pull out the data from Ethereum blockchain.
You will be interactively asked for this API key.

### Setup the notebook

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

from eth_defi.uniswap_v3.constants import UNISWAP_V3_FACTORY_CREATED_AT_BLOCK
from eth_defi.uniswap_v3.events import fetch_events_to_csv

/Users/moo/code/executor/trade-executor/deps/web3-ethereum-defi/eth_defi/uniswap_v3/events.py:15: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


### Download the raw data

For simplicity, you can sign up for free access to an Ethereum node for example at [Infura](https://infura.io/), however we recommend you to [run your own Ethereum node](https://tradingstrategy.ai/blog/preparing-a-server-for-hosting-goethereum-full-node)/

In [3]:
# Get your node JSON-RPC URL
json_rpc_url = input("Please enter your Ethereum mainnet JSON-RPC URL here")

As an example, here we download raw events from first 1000000 blocks after Uniswap V3 factory is created, the events will be stored in several CSV files at `/tmp` folder.

The scan will take hours. It can resumed from crash, as we save the last scanned block in a JSON state file.

In [ ]:
from eth_defi.event_reader.json_state import JSONFileScanState

start_block = UNISWAP_V3_FACTORY_CREATED_AT_BLOCK
end_block = UNISWAP_V3_FACTORY_CREATED_AT_BLOCK + 1_000_000

state = JSONFileScanState("/tmp/price-scan.json")

fetch_events_to_csv(json_rpc_url, state, start_block=start_block, end_block=end_block)

Starting to read block range 12,386,121 - 13,369,621


  0%|          | 0/983500 [00:00<?, ?it/s]

### Analyse the pool's price

In Uniswap V3, you can get the price of any pool from any given moment using only swap events

In [ ]:
swap_df = pd.read_csv("/tmp/uniswapv3-Swap.csv")

Choose a pool to analyse, for example: USDC/ETH 0.3%

In [ ]:
pool_address = "0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8"

Extract only swaps related to this pool from the dataframe

In [ ]:
df = swap_df[swap_df.pool_contract_address == pool_address]
df

In Uniswap V3, we can easily get pool's price from pool's tick

In [ ]:
def tick_to_price(tick):
    return 1.0001**tick

df["price"] = df.apply(lambda row: tick_to_price(row["tick"]), axis=1)

Then we can plot the price of this pool in first 1000000 blocks

In [ ]:
plt.rcParams["figure.dpi"] = 200

df.index = df.timestamp
df["price"].plot()

plt.xticks(rotation=45, ha="right")